# Telco Customer Churn para ICP4D

Usaremos este 'notebook' para crear un modelo de 'machine learning' para predecir el 'CHURN' en los clientes.

# 1.0 Instalar las paqueterías requeridas

In [ ]:
!pip install --user watson-machine-learning-client --upgrade | tail -n 1
!pip install --user pyspark==2.3.3 --upgrade|tail -n 1

# 2.0 Cargar y limpiar los datos.
Cargamos nuestros datos como un marco de datos (Data Frame) de pandas.

* Resalta la celda inferior dándole clic.
* Da clic en el ícono `01/00` "Find and add data" en la parte superior derecha de la 'Notebook'.
* Si estás usando 'Virtualized data', comienza seleccionando la pestaña 'Files'. Ahora, selecciona tus datos virtualizados (ej. MYSCHEMA.BILLINGPRODUCTCUSTOMERS), da clic en 'Insert to code' y selecciona 'Insert Pandas DataFrame'.
* Si estás usando esta 'Notebook' sin datos virtualizados, agrega localmente el archivo: `Telco-Customer-Churn.csv` Seleccionando la pestaña 'Files'. Después, selecciona el archivo: 'Telco-Customer-Churn.csv'. Da clic en 'Insert to code' y selecciona 'Insert Pandas DataFrame'.
* El códico para traer los datos a este ambiente de 'Notebook' y crear el marco de datos de Pandas (Pandas DataFrame) será agregado el la celda inferior.
* Correr la celda.


In [ ]:
import pandas as pd
# Coloca el cursor debajo e inserta el marco de datos de pandas(Pandas DataFrame) para los datos de la Telco.



Usaremos la convención de nombramiento de Pandas 'df' para nuestro marco de datos (DataFrame). Asegúrate de que la celda inferior use el mismo nombre para el marco de datos(DataFrame) usado en la celda superior. Para el archivo cargado de forma local, debe verse como: 'df_data_1' o 'df_data_2' o 'df_data_x'. Para el caso de los datos virtualizados, debe verse como: data_df_1 o data_df_2 o data_df_x.

In [ ]:
# Para datos virtualizados:
# df = data_df_1

# Para carga local:
df = df_data_1

### 2.1 Desplegar la característica 'CustomerID' (columna)

In [ ]:
df = df.drop('ClienteID', axis=1)
df.head(5)

### 2.2 Examinar los tipos de datos de las características (columnas).

In [ ]:
df.info()

### 2.3 Verificar la necesidad de convertir la columna 'TotalCharges' a numérico si es detectado como objeto

Si el 'df.info' superior, muestra la columna "TotalCharges" como un objeto, necesitamos convertirlo a numérico. Si ya lo has hecho durante el ejericio previo para "Visualización de datos con refinería de datos", puedes saltar este paso `2.5`.

In [ ]:
totalCharges = df.columns.get_loc("Cargos totales")
print(totalCharges)

In [ ]:
new_col = pd.to_numeric(df.iloc[:, totalCharges], errors='coerce')
new_col

### 2.4 Modificamos nuestro marco de datos 'Data Frame' para reflejar el nuevo tipo de dato

In [ ]:
df.iloc[:, totalCharges] = pd.Series(new_col)
df



### 2.5 Los valores NaN deben ser removidos para crear un modelo más certero.

In [ ]:
# Revisar si tenemos valores 'NaN' ('Not a Number').
df.isnull().values.any()

Configura la columna 'nan_column' al numero de columna de 'TotalCharges' (comenzando en 0).

In [ ]:
nan_column = df.columns.get_loc("Cargos totales")
print(nan_column)

In [ ]:
# Maneja los valores faltantes para la columna 'nan_column' de 'Cargos totales'

import numpy as np
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy='mean')

df.iloc[:, nan_column] = imp.fit_transform(df.iloc[:, nan_column].values.reshape(-1, 1))
df.iloc[:, nan_column] = pd.Series(df.iloc[:, nan_column])

In [ ]:
# Verificar si tenemos valores 'NaN'.
df.isnull().values.any()

### 2.6 Visualizar los datos

In [ ]:
import json
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, StandardScaler
import sklearn.feature_selection
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn import metrics

In [ ]:
# Conteo de frecuencia en la permanencia de la trama (plot tenure).
sns.set(style="darkgrid")
sns.set_palette("hls", 3)
fig, ax = plt.subplots(figsize=(20,10))
ax = sns.countplot(x="Permanencia", hue="CHURN", data=df)

In [ ]:
# Conteo de frecuencia en la permanencia de la trama (plot tenure).
sns.set(style="darkgrid")
sns.set_palette("hls", 3)
fig, ax = plt.subplots(figsize=(20,10))
ax = sns.countplot(x="Contrato", hue="CHURN", data=df)

In [ ]:
# Conteo de frecuencia en la permanencia de la trama (plot tenure). 
sns.set(style="darkgrid")
sns.set_palette("hls", 3)
fig, ax = plt.subplots(figsize=(20,10))
ax = sns.countplot(x="Soporte tecnico", hue="CHURN", data=df)


In [ ]:
# Crear la cuadrícula para relaciones en pares.
gr = sns.PairGrid(df, height=5, hue="CHURN")
gr = gr.map_diag(plt.hist)
gr = gr.map_offdiag(plt.scatter)
gr = gr.add_legend()

In [ ]:
# Configurar el tamaño de la trama.
fig, ax = plt.subplots(figsize=(6,6))

# Distribución de atributos
a = sns.boxplot(orient="v", palette="hls", data=df.iloc[:, totalCharges], fliersize=14)


In [ ]:
# Distribución de datos de cargas totales.
histogram = sns.distplot(df.iloc[:, totalCharges], hist=True)
plt.show()

In [ ]:
tenure  = df.columns.get_loc("Permanencia")
print(tenure)

In [ ]:
# Distribución de permanencia de datos (Tenure).
histogram = sns.distplot(df.iloc[:, tenure], hist=True)
plt.show()

In [ ]:
monthly = df.columns.get_loc("Cargos mensuales")
print(monthly)

In [ ]:
# Distribución de datos de cargos mensuales.
histogram = sns.distplot(df.iloc[:, monthly], hist=True)
plt.show()


Entender la distribución de datos


# 3.0 Crear un modelo

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
import json

spark = SparkSession.builder.getOrCreate()
df_data = spark.createDataFrame(df)
df_data.head()

### 3.1 Dividir los datos en conjuntos de entrenamiento y de prueba.

In [ ]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

print("Número de registros para entrenamiento: " + str(train_data.count()))
print("Número de registros para evaluación: " + str(test_data.count()))

### 3.2 Examinar el esquema de marco de datos (DataFrame Schema) de 'Spark'.
Observa los tipos de datos para determinar los requerimientos para la ingeniería de características.

In [ ]:
spark_df.printSchema()

### 3.3 Usar 'StringIndexer' para codificar una columna de etiquetas 'string' a una columna de etiquetas índice.

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model


si_gender = StringIndexer(inputCol = 'Genero', outputCol = 'gender_IX')
si_Partner = StringIndexer(inputCol = 'Pareja', outputCol = 'Partner_IX')
si_Dependents = StringIndexer(inputCol = 'Dependientes', outputCol = 'Dependents_IX')
si_PhoneService = StringIndexer(inputCol = 'Servicio telefonico', outputCol = 'PhoneService_IX')
si_MultipleLines = StringIndexer(inputCol = 'Lineas multiples', outputCol = 'MultipleLines_IX')
si_InternetService = StringIndexer(inputCol = 'Servicio de internet', outputCol = 'InternetService_IX')
si_OnlineSecurity = StringIndexer(inputCol = 'Seguridad en linea', outputCol = 'OnlineSecurity_IX')
si_OnlineBackup = StringIndexer(inputCol = 'Respaldo en linea', outputCol = 'OnlineBackup_IX')
si_DeviceProtection = StringIndexer(inputCol = 'Proteccion de dispositivo', outputCol = 'DeviceProtection_IX')
si_TechSupport = StringIndexer(inputCol = 'Soporte tecnico', outputCol = 'TechSupport_IX')
si_StreamingTV = StringIndexer(inputCol = 'StreamingTV', outputCol = 'StreamingTV_IX')
si_StreamingMovies = StringIndexer(inputCol = 'StreamingMovies', outputCol = 'StreamingMovies_IX')
si_Contract = StringIndexer(inputCol = 'Contrato', outputCol = 'Contract_IX')
si_PaperlessBilling = StringIndexer(inputCol = 'Facturacion sin papel', outputCol = 'PaperlessBilling_IX')
si_PaymentMethod = StringIndexer(inputCol = 'Metodo de pago', outputCol = 'PaymentMethod_IX')


In [ ]:
si_Label = StringIndexer(inputCol="CHURN", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

### 3.4 Crear un vector.

In [ ]:
va_features = VectorAssembler(inputCols=['gender_IX',  'Adulto Mayor', 'Partner_IX', 'Dependents_IX', 'PhoneService_IX', 'MultipleLines_IX', 'InternetService_IX', \
                                         'OnlineSecurity_IX', 'OnlineBackup_IX', 'DeviceProtection_IX', 'TechSupport_IX', 'StreamingTV_IX', 'StreamingMovies_IX', \
                                         'Contract_IX', 'PaperlessBilling_IX', 'PaymentMethod_IX', 'Cargos totales', 'Cargos mensuales'], outputCol="features")

### 3.5 Crear una 'pipeline', y ajustar un modelo usando 'RandomForestClassifier'. 
Montar todas las etapas a una 'pipeline'. No esperamos una regresión lineal limpia, así que usaremos 'RandomForestClassifier' para encontrar el mejor árbol de decisión para nuestros datos.

In [ ]:
classifier = RandomForestClassifier(featuresCol="features")

pipeline = Pipeline(stages=[si_gender, si_Partner, si_Dependents, si_PhoneService, si_MultipleLines, si_InternetService, si_OnlineSecurity, si_OnlineBackup, si_DeviceProtection, \
                            si_TechSupport, si_StreamingTV, si_StreamingMovies, si_Contract, si_PaperlessBilling, si_PaymentMethod, si_Label, va_features, \
                            classifier, label_converter])

model = pipeline.fit(train_data)

In [ ]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

#La evauación predeterminada es 'areaUnderROC'
print("areaUnderROC = %g" % area_under_curve)

# 4.0 Guardar el modelo y probar los datos.

Agrega un nombre único para el MODEL_NAME.

In [ ]:
MODEL_NAME = "mi-modelo mi-fecha"

### 4.1 Guardar el modelo en 'ICP4D local Watson Machine Learning'.
Reemplaza el URL, apikey e instance ID con tus credenciales de 'Watson Machine Learning'.


In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_credentials = {
                   "url": "*****",
                   "apikey" : "*****",
                   "instance_id": "*****",
                  }

client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
client.repository.list()
client.repository.list_models()
client.deployments.list()


In [ ]:
# Guardar el modelo
model_props = model_props = {client.repository.ModelMetaNames.NAME: MODEL_NAME}
published_model = client.repository.store_model(model=model, pipeline=pipeline, meta_props=model_props, training_data=train_data)

In [ ]:
# Usar esta celda para cualquier limpieza de modelos y despliegues previos.
client.repository.list_models()
client.deployments.list()

# client.repository.delete('GUID del modelo guardado')
# client.deployments.delete('GUID del modelo desplegado')


### 4.2 Escribe los datos de prueba sin ninguna etiqueta a un .csv, para usarlo después como puntuación por lotes.

In [ ]:
from project_lib import Project
project = Project(""
,
"Project_ID" ,
"Access_Token" )
write_score_CSV=test_data.toPandas().drop(['Churn'], axis=1)
project.save_data(file_name = 
'TelcoCustomerSparkMLBatchScore.csv' ,data = write_score_CSV.to_csv(index=False))

### 4.3 Escribe los datos de prueba a un .csv para usarlos posteriormente para la evaluación.

In [ ]:
from project_lib import Project
project = Project(""
,
"Project_ID" ,
"Access_Token" )
write_eval_CSV=test_data.toPandas()
project.save_data(file_name = 
'TelcoCustomerSparkMLEval.csv' ,data = write_eval_CSV.to_csv(index=False))

In [ ]:
## Felicidades, ya has creado un modelo basado en los datos de 'CHURN' de clientes, y lo has desplegado a 'Watson Machine Learning'!